In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [2]:
# Load the train data

train_df = pd.read_csv('data/train.csv')
train_df.head(5)

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392


In [3]:
# There are 51K images in train set
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51033 entries, 0 to 51032
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   image          51033 non-null  object
 1   species        51033 non-null  object
 2   individual_id  51033 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [4]:
# Sample submission
sample_sub = pd.read_csv('data/sample_submission.csv')
sample_sub.head(5)

,image,predictions
0,000110707af0ba.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
1,0006287ec424cb.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
2,000809ecb2ccad.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
3,00098d1376dab2.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
4,000b8d89c738bd.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...


In [5]:
#Size of train set
train_df.shape

(51033, 3)

In [6]:
# # Load some images
# from tqdm.autonotebook import tqdm
# from keras.preprocessing import image
# from keras.applications.imagenet_utils import preprocess_input

# # Loading images 
# # ds = training dataframe dataset , s = size of train_df, d = 'path to train_df'

# def load_images(ds, s , d):
#     x_train = np.zeros((s, 32, 32, ds.shape[1]))
#     c = 0
#     for fig in tqdm(ds['image']):
#         img = image.load_img("../input/happy-whale-and-dolphin/"+
#                              d+"/"+fig, 
#                              target_size=(32, 32, 3))
#         x = image.img_to_array(img)
#         x = preprocess_input(x)
#         x_train[c] = x
#         c += 1
#     return x_train
# X = load_images(train_df, train_df.shape[0], 'train_images')
# X /= 255
                         

In [21]:
train_df['class'] = train_df['species'].apply(lambda x: x.split('_')[-1])

In [22]:
train_df.head()

,image,species,individual_id,class
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,whale
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,whale
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,whale
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,dolphin
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,whale


In [23]:
train_df['class'].value_counts()

whale      26479
dolphin    15878
beluga      7443
dolpin      1117
globis       116
Name: class, dtype: int64

In [24]:
# Load images in training data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   height_shift_range = 0.1,
                                   width_shift_range = 0.1,
                                validation_split = 0.2,
                                horizontal_flip = True)

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

train_dir = 'data/train_images/'

train_set = train_datagen.flow_from_dataframe(train_df, train_dir,
                                              seed = 101,
                                            target_size = (64,64),
                                            batch_size = 32,
                                            x_col='image',
                                            y_col='class',
                                            class_mode = 'categorical',
                                            subset = 'training')

valid_set = valid_datagen.flow_from_dataframe(train_df, 
                                              train_dir,
                                                seed = 101,
                                                target_size = (64,64),
                                              x_col='image',
                                                y_col='class',
                                                batch_size = 32,
                                                class_mode = 'categorical',
                                                subset = 'validation')

Found 40827 validated image filenames belonging to 5 classes.
Found 10206 validated image filenames belonging to 5 classes.


In [25]:
valid_set

In [26]:
# '''
# Encode individual id's present in training data using 
# one hot encoding and label encoding
# Writing a function to retrurn labels for individual image id's
# ie: train_df['individual_id']

# '''
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# def labels(l):
#     values = np.array(l)
#     le = LabelEncoder()
#     oe = OneHotEncoder(parse = False)
#     le_values = le.fit_transform(values)
#     oe_values = oe.fit_transform(values)
#     return le_values, oe_values

# y, le = labels(train_df['individual_id'])


# Build the Model

In [27]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras import Sequential

model = Sequential()

model.add(Conv2D(32, (6,6), strides = (1,1), input_shape =
                (64,64,3)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32, (3,3), strides = (1,1)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.85))

model.add(Dense(5, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 59, 59, 32)        3488      
                                                                 
 batch_normalization_4 (Batc  (None, 59, 59, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 59, 59, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 29, 29, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 27, 27, 32)        9248      
                                                                 
 batch_normalization_5 (Batc  (None, 27, 27, 32)      

In [28]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(patience = 3,
                          monitor = 'accuracy',
                          mode = 'max')]

In [ ]:
hist = model.fit(train_set,
                 epochs = 10,
                 validation_data = valid_set,
                 batch_size = 40000,
                 verbose = 1,
                callbacks = callbacks)


Epoch 1/10
1276/1276 [==============================] - 729s 571ms/step - loss: 1.0824 - accuracy: 0.5600 - val_loss: 0.9707 - val_accuracy: 0.5906
Epoch 2/10
1276/1276 [==============================] - 721s 565ms/step - loss: 0.8609 - accuracy: 0.6114 - val_loss: 0.9472 - val_accuracy: 0.6555
Epoch 3/10
1276/1276 [==============================] - 715s 560ms/step - loss: 0.7732 - accuracy: 0.6644 - val_loss: 0.6521 - val_accuracy: 0.7190
Epoch 4/10
1276/1276 [==============================] - 716s 561ms/step - loss: 0.7096 - accuracy: 0.6949 - val_loss: 0.6010 - val_accuracy: 0.7563
Epoch 5/10
1276/1276 [==============================] - 715s 560ms/step - loss: 0.6694 - accuracy: 0.7188 - val_loss: 0.7587 - val_accuracy: 0.7060
Epoch 6/10
 134/1276 [==>...........................] - ETA: 8:18 - loss: 0.6397 - accuracy: 0.7323

# Model Evaluation

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (15,5))
plt.plot(hist.history['accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:

plt.figure(figsize = (15,5))
plt.plot(hist.history['loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
print(hist.history['accuracy'])
print(hist.history['val_accuracy'])
print(hist.history['loss'])